In [6]:
!pip install sentence-transformers


In [7]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the CSV
df = pd.read_csv("/kaggle/input/shl-assessments/shl_assessments.csv")

# Initialize model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings from descriptions
df['embedding'] = df['Description'].apply(lambda desc: model.encode(desc, convert_to_numpy=True))


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

def get_top_assessments(query, top_k=5):
    query_embedding = model.encode(query, convert_to_numpy=True)

    # Calculate cosine similarity
    similarities = df['embedding'].apply(lambda emb: cosine_similarity([query_embedding], [emb])[0][0])
    df['score'] = similarities

    # Return top matches
    top_matches = df.sort_values(by='score', ascending=False).head(top_k)
    return top_matches[['Assessment Name', 'URL', 'Remote Testing Support', 'Adaptive/IRT Support', 'Duration', 'Test Type', 'score']]


In [9]:
query = "I want to test Python coding skills for developers"
results = get_top_assessments(query)
results


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,Assessment Name,URL,Remote Testing Support,Adaptive/IRT Support,Duration,Test Type,score
3,Coding Simulation – Python,https://www.shl.com/en/assessments/technical/c...,Yes,No,45 minutes,Technical,0.870351
0,Verify G+,https://www.shl.com/en/assessments/cognitive/v...,Yes,Yes,36 minutes,Cognitive Ability,0.351872
2,Occupational Personality Questionnaire,https://www.shl.com/en/assessments/personality...,Yes,Yes,30 minutes,Personality,0.351719
4,Language Proficiency – English,https://www.shl.com/en/assessments/language/en...,Yes,No,20 minutes,Language,0.327843
1,Business Personality Questionnaire,https://www.shl.com/en/assessments/personality...,Yes,No,25 minutes,Personality,0.156058


In [15]:
!pip install streamlit sentence-transformers pandas scikit-learn


In [17]:
import streamlit as st
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

st.set_page_config(page_title="SHL Assessment Recommender", layout="wide")
st.title("🔍 SHL Assessment Recommender")

@st.cache_resource
def load_model():
    return SentenceTransformer("all-MiniLM-L6-v2")

@st.cache_data
def load_data():
    df = pd.read_csv("/kaggle/input/shl-assessments/shl_assessments.csv")
    model = load_model()
    df["embedding"] = df["Description"].apply(lambda x: model.encode(x, convert_to_numpy=True))
    return df

df = load_data()
model = load_model()

query = st.text_area("Enter a job description or query:", height=150)

if st.button("🔎 Recommend"):
    if query.strip():
        query_embedding = model.encode(query, convert_to_numpy=True)
        df["score"] = df["embedding"].apply(lambda emb: cosine_similarity([query_embedding], [emb])[0][0])
        top_matches = df.sort_values(by="score", ascending=False).head(10)
        
        st.markdown("### 📝 Recommended Assessments:")
        st.dataframe(top_matches[[
            "Assessment Name", "URL", "Remote Testing Support",
            "Adaptive/IRT Support", "Duration", "Test Type"
        ]], use_container_width=True)
    else:
        st.warning("Please enter a query above.")


2025-04-06 12:13:12.044 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 12:13:12.046 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 12:13:12.047 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 12:13:12.049 No runtime found, using MemoryCacheStorageManager
2025-04-06 12:13:12.051 No runtime found, using MemoryCacheStorageManager
2025-04-06 12:13:12.052 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 12:13:12.053 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 12:13:12.054 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 12:13:12.556 Thread 'Thread-7': missing ScriptRunContext! This warning can be ignored

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-04-06 12:13:13.106 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 12:13:13.106 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 12:13:13.108 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 12:13:13.108 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 12:13:13.109 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 12:13:13.111 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 12:13:13.112 Session state does not function when running a script without `streamlit run`
2025-04-06 12:13:13.113 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-06 12:13

In [20]:
streamlit_code = """
import streamlit as st
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

st.set_page_config(page_title="SHL Assessment Recommender", layout="wide")
st.title("🔍 SHL Assessment Recommender")

@st.cache_resource
def load_model():
    return SentenceTransformer("all-MiniLM-L6-v2")

@st.cache_data
def load_data():
    df = pd.read_csv("shl_assessments.csv")
    model = load_model()
    df["embedding"] = df["Description"].apply(lambda x: model.encode(x, convert_to_numpy=True))
    return df

df = load_data()
model = load_model()

query = st.text_area("Enter a job description or query:", height=150)

if st.button("🔎 Recommend"):
    if query.strip():
        query_embedding = model.encode(query, convert_to_numpy=True)
        df["score"] = df["embedding"].apply(lambda emb: cosine_similarity([query_embedding], [emb])[0][0])
        top_matches = df.sort_values(by="score", ascending=False).head(10)
        
        st.markdown("### 📝 Recommended Assessments:")
        st.dataframe(top_matches[[
            "Assessment Name", "URL", "Remote Testing Support",
            "Adaptive/IRT Support", "Duration", "Test Type"
        ]], use_container_width=True)
    else:
        st.warning("Please enter a query above.")
"""

with open("streamlit_app.py", "w") as f:
    f.write(streamlit_code)

print("✅ streamlit_app.py saved.")


✅ streamlit_app.py saved.


In [ ]:
!streamlit run streamlit_app.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://34.44.215.48:8501

